In [84]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [85]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,NaN,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,NaN,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,NaN,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig (WA)/Subiaco (WAFL),7.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1990-06-07,84.0,0.0,0.0,0.0,0.0,4.0,1.0,50.

In [86]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

In [87]:
match = player_stats[player_stats['Match_ID'] == "202323_WesternBulldogs_WestCoast"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
27805,202323_WesternBulldogs_WestCoast,West Coast,Alex Witherden,202323,Alex_Witherden,MEDIUM_DEFENDER,CD_I998128,24,188,85,23,RIGHT,VIC,2016.0,2017.0,"St Joseph&apos;s, Geel (Vic)/Geel College (Vic...",23.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.000000,-0.267964,0.023892,1.114284,0.536018,0.578265,8.365421,5.327914,3.037508,-1.010768,0.023430,1.118252,5.976717,Alex Witherden_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs
27806,202323_WesternBulldogs_WestCoast,West Coast,Andrew Gaff,202323,Andrew_Gaff,MIDFIELDER,CD_I290801,31,184,83,3,LEFT,VIC,2010.0,2011.0,Kew (Vic)/Carey Grammar (Vic)/Oakleigh U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023,2.148563,0.191014,0.052414,0.631304,0.340834,0.290469,4.194502,3.959685,0.234816,-0.230022,0.138102,0.953913,7.855344,Andrew Gaff_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs
27807,202323_WesternBulldogs_WestCoast,West Coast,Bailey J. Williams,202323,Bailey_J._Williams,RUCK,CD_I1006114,23,201,101,32,RIGHT,VIC,2018.0,2020.0,Pines (Vic)/Western Port SC (Vic)/Dandenong U18,35.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ruck,FINAL_TEAM,Ruck,Ruck,NaN,NaN,2023,0.000000,0.251066,0.067486,0.449135,0.451447,-0.002312,4.723950,4.275080,0.448870,-0.014674,0.022624,-0.076412,3.294606,Bailey J. Williams_2023,NaN,NaN,Western Bulldogs,West Coast,Western Bulldogs
27808,202323_WesternBulldogs_WestCoast,West Coast,Campbell Chesser,202323,Campbell_Chesser,MIDFIELD

In [88]:
match_for = match.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_for.columns = [x+"_F" for x in list(match_for)]
match_against = match.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
match_against.columns = [x+"_A" for x in list(match_against)]
match_summary = pd.concat([match_for, match_against], axis=1)
for col in [x.replace("_F", "") for x in list(match_for)]:
        match_summary[col+"_diff"] = match_summary[col+"_F"] - match_summary[col+"_A"]
match_summary

,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_diff,xScore_diff,vaep_value_diff,vaep_value_received_diff,offensive_value_diff,defensive_value_diff,exp_vaep_value_diff,exp_vaep_value_received_diff,exp_offensive_value_diff,exp_defensive_value_diff
West Coast,0.0,83.056271,18.050087,14.241962,16.105287,1.944800,103.830771,90.770835,98.886060,4.944711,0.0,82.120013,16.107976,14.018544,14.393552,1.714424,104.667123,95.467977,96.016119,8.651004,0.0,0.936258,1.942111,0.223419,1.711735,0.230376,-0.836352,-4.697142,2.86994,-3.706293
Western Bulldogs,0.0,82.120013,16.107976,14.018544,14.393552,1.714424,104.667123,95.467977,96.016119,8.651004,0.0,83.056271,18.050087,14.241962,16.105287,1.944800,103.830771,90.770835,98.886060,4.944711,0.0,-0.936258,-1.942111,-0.223419,-1.711735,-0.230376,0.836352,4.697142,-2.86994,3.706293


In [89]:
home_team = match[match['Team'] == match['Home_Team']]
away_team = match[match['Team'] != match['Home_Team']]

home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
home_team['player_value'] = home_team['exp_vaep_value_%'] * (home_team['xScore'].sum())

away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
away_team['player_value'] = away_team['exp_vaep_value_%'] * (away_team['xScore'].sum())
match_player_values = pd.concat([home_team, away_team])
match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'exp_vaep_value_%', 'player_value']].sort_values(by='player_value', ascending=False).head(20)

,Player,Team,Position_Group,Score,xScore,exp_vaep_value,exp_vaep_value_%,player_value
27845,Marcus Bontempelli,Western Bulldogs,Midfield,NaN,3.469234,20.289246,0.193845,15.918591
27848,Rory Lobb,Western Bulldogs,Forward,NaN,17.116590,11.169317,0.106713,8.763252
27810,Elliot Yeo,West Coast,NaN,NaN,3.583849,10.120924,0.097475,8.095926
27838,Ed Richards,Western Bulldogs,Back,NaN,0.000000,10.116849,0.096657,7.937505
27816,Jayden Hunt,West Coast,Midfield,NaN,5.764310,9.682959,0.093257,7.745588
27822,Oscar Allen,West Coast,Forward,NaN,21.823077,9.266260,0.089244,7.412263
27825,Tim Kelly,West Coast,Midfield,NaN,3.229225,9.131614,0.087947,7.304557
27805,Alex Witherden,West Coast,Back,NaN,0.000000,8.365421,0.080568,6.691665
27829,Adam Treloar,Western Bulldogs,Midfield,NaN,8.975932,7.646901,0.073059,5.999626
27817,Jeremy McGovern,West Coast,Back,NaN,0.000000,7.365036,0.070933,5.891437


In [92]:
def create_player_value(match_player_stats):
    
    home_team = match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']]
    home_team['exp_vaep_value_%'] = home_team['exp_vaep_value'] / home_team['exp_vaep_value'].sum()
    home_team['player_value'] = home_team['exp_vaep_value_%'] * home_team['xScore'].sum()

    away_team = match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']]
    away_team['exp_vaep_value_%'] = away_team['exp_vaep_value'] / away_team['exp_vaep_value'].sum()
    away_team['player_value'] = away_team['exp_vaep_value_%'] * away_team['xScore'].sum()
    match_player_values = pd.concat([home_team, away_team])
    match_player_values[['Player', 'Team', 'Position_Group', 'Score', 'xScore', 'exp_vaep_value', 'player_value']].sort_values(by='player_value', ascending=False).head(10)

    return match_player_values

In [93]:
match_player_values_list = []
for match_id in list(player_stats['Match_ID'].unique()):
    
    match_player_stats = player_stats[player_stats['Match_ID'] == match_id]
    match_player_values = create_player_value(match_player_stats)
    match_player_values_list.append(match_player_values)
    
player_values = pd.concat(match_player_values_list)


In [94]:
player_values_2023 = player_values[player_values['Season'] == 2023]
player_values_2023.sample(5)

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,exp_vaep_value_%,player_value
23699,202312_WesternBulldogs_Geelong,Western Bulldogs,Marcus Bontempelli,202312,Marcus_Bontempelli,MIDFIELDER,CD_I297373,27,194,96,4,LEFT,VIC,2013.0,2014.0,Eltham (Vic)/Northern U18,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1995-11-24,79.0,2.0,0.0,19.0,2.0,4.0,0.0,0.0,0.0,0.0,45.8,11.0,0.0,0.0,0.0,8.0,69.6,23.0,108.0,16.0,11.0,1.0,2.0,1.0,0.0,1.0,0.0,9.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,61.1,0.0,0.0,18.0,3.6,5.0,1.0,0.0,513.0,1.0,19.0,14.3,3.0,0.0,8.0,1.0,2.0,1.0,2.0,8.0,0.0,4.0,24.0,4.0,13.0,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,NaN,NaN,2023,7.001830,0.065671,0.013824,1.385183,1.278146,0.107037,15.962914,15.888789,0.074125,0.028674,0.008430,1.856161,11.913538,Marcus Bontempelli_2023,2.0,-5.001830,Western Bulldogs,Geelong,Geelong,0.138159,12.824304
24729,202316_Essendon_PortAdelaide,Essendon,Jayden Laverde,202316,Jayden_Laverde,KEY_DEFENDER,CD_I298280,27,193,92,15,RIGHT,VIC,2014.0,2015.0,Keilor (Vic)/PEGS (Vic)/Western U18,20.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1996-04-12,85.0,0.0,0.0,0.0,0.0,0.0,2.0,66.7,3.0,1.0,40.0,6.0,0.0,0.0,0.0,8.0,80.0,15.0,75.0,12.0,7.0,0.0,0.0,2.0,100.0,0.0,1.0,3.0,7.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,87.5,0.0,0.0,8.0,1.1,3.0,0.0,0.0,283.0,4.0,9.0,14.7,5.0,0.0,2.0,1.0,1.0,4.0,1.0,5.0,0.0,1.0,15.0,0.0,9.0,NaN,NaN,Centre Half Back,FINAL_TEAM,Key-Back,Back,NaN,NaN,2023,2.370568,-0.204148,0.004549,1.328709,0.702227,0.626482,2.934035,1.227732,1.706303,-0.252059,0.129918,0.093150,0.068389,Jayden Laverde_2023,6.0,3.629432,Essendon,Port Adelaide,Port Adelaide,0.024048,1.880603
22259,202309_Carlton_WesternBulldogs,Western Bulldogs,Alex Keath,202309,Alex_Keath,KEY_DEFENDER,CD_I290797,31,198,93,42,LEFT,VIC,2017.0,2017.0,Shepparton Bears (Vic)/Murray U18/Gold Coast L...,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1992-01-20,88.0,0.0,0.0,0.0,0.0,1.0,2.0,50.0,4.0,0.0,22.2,2.0,0.0,0.0,0.0,4.0,88.9,9.0,42.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,85.7,0.0,0.0,7.0,3.5,3.0,0.0,0.0,185.0,6.0,5.0,2.8,2.0,0.0,1.0,0.0,0.0,5.0,1.0,2.0,0.0,1.0,9.0,2.0,7.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023,0.000000,-0.048323,0.003367,0.716158,0.196034,0.520124,4.415189,2.956961,1.458227,-0.087397,-0.044084,0.936411,6.120813,Alex Keath_2023,0.0,0.000000,Carlton,Western Bulldogs,Carlton,0.040062,2.685428
2

In [96]:
total_player_values = player_values_2023.groupby(['Player', 'Team']).sum()[['exp_vaep_value', 'player_value', 'xScore']].reset_index().sort_values(by='player_value', ascending=False)
total_player_values.head(10)
# total_player_values[total_player_values['Player'] == "Nick Daicos"]

,Player,Team,exp_vaep_value,player_value,xScore
417,Marcus Bontempelli,Western Bulldogs,285.512198,199.372368,128.733900
105,Charlie Curnow,Carlton,245.467148,184.122672,420.047065
565,Shai Bolton,Richmond,251.072148,180.792473,204.606629
110,Christian Petracca,Melbourne,238.859557,176.546741,218.257694
483,Noah Anderson,Gold Coast,241.456458,167.099368,103.656513
647,Zak Butters,Port Adelaide,216.604320,156.001636,71.230558
117,Connor Rozee,Port Adelaide,213.654308,155.228108,124.859093
577,Taylor Walker,Adelaide,190.938307,139.813431,346.683093
570,Stephen Coniglio,Greater Western Sydney,187.010688,139.150210,76.622984
323,Jordan Dawson,Adelaide,198.003513,137.767640,70.301705
